In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from tensorflow.keras.models import Sequential

In [ ]:
print(f"TensorFlow Devices:\n{tf.config.list_physical_devices()}")
print(f"TensorFlow Version: {tf.__version__}")

## Load data

In [ ]:
dataset = tf.keras.datasets.mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = dataset.load_data()

In [ ]:
scale = x_train.max()
x_train_scale = x_train / scale
x_test_scale = x_test / scale

In [ ]:
mask_train = y_train < 2
mask_test = y_test < 2
x_train_01 = x_train_scale[mask_train]
y_train_01 = y_train[mask_train]
x_test_01 = x_test_scale[mask_test]
y_test_01 = y_test[mask_test]

## Train binary model from scratch

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
metrics = [tf.keras.metrics.BinaryAccuracy(name='accuracy'),
           tf.keras.metrics.Precision(name='precision'),
           tf.keras.metrics.Recall(name='recall'),
           tf.keras.metrics.AUC(name='auc')]

In [ ]:
def cnn_2d():

    model = Sequential()
    model.add(Conv2D(filters=8, kernel_size=3, padding='same', activation='relu', input_shape=(28,28,1)))
    model.add(MaxPool2D(pool_size=2))
    model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPool2D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='Adam', loss=loss, metrics=metrics)
              
    return model

In [ ]:
model = cnn_2d()

In [ ]:
model.summary()

In [ ]:
model_history = model.fit(x_train_01, y_train_01, epochs=5, verbose=1, validation_split=0.1, batch_size=32)

## Evaluate model

In [ ]:
metrics_names = ['loss', 'accuracy', 'precision', 'recall', 'auc']
ms = ['^', 's', 'p', 'o']
epochs = np.arange(5)
fig, axs = plt.subplots(1,2,figsize=[10,5])
axs[0].set_title('loss')
axs[0].plot(model_history.history['loss'], label='train')
axs[0].plot(model_history.history['val_loss'], label='val')
axs[0].legend()
axs[1].set_title('metrics')
for i, name in enumerate(metrics_names[1:]):
    axs[1].scatter(epochs, model_history.history[name], label=name, marker=ms[i], color='C0')
    axs[1].plot(model_history.history[name], alpha=0.5, color='C0')
    axs[1].scatter(epochs, model_history.history[f'val_{name}'], marker=ms[i], color='C1')
    axs[1].plot(model_history.history[f'val_{name}'], alpha=0.5, color='C1')
axs[1].legend()
plt.tight_layout()    

In [ ]:
model.evaluate(x=x_test_01, y=y_test_01)

In [ ]:
y_pred = model.predict(x_test_01).flatten().round().astype(int)

In [ ]:
(y_pred == y_test_01).sum() / y_pred.shape[0]

In [ ]:
weights = model.get_weights()

In [ ]:
conv1_filter = np.transpose(weights[0].reshape(3, 3, 8), axes=(2, 0, 1))

In [ ]:
vmin = conv1_filter.min()
vmax = conv1_filter.max()

fig, axs = plt.subplots(2,4)
for i in range (2):
    for j in range (4):
        ind = i*4+j
        axs[i, j].imshow(conv1_filter[ind], vmin=vmin, vmax=vmax)
plt.tight_layout()

## ResNet50

In [ ]:
x_train_res = np.zeros((x_train.shape[0], 32, 32))
x_train_res[:, 2:30, 2:30] = x_train

In [ ]:
model = tf.keras.applications.ResNet50(include_top=True, weights=None, input_shape=(32,32,1), classes=10)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=["accuracy"])

In [ ]:
# can only train one model per jupyter kernel or else it dies
#model.fit(x_train_res, y_train, epochs=1, batch_size=128)